In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [3]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [4]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [5]:
dimension = 2

In [6]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [7]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [8]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    # v = torch.ones(u_hat.shape)
    # ux = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = v, create_graph = True)[0]
    
    ux = torch.zeros(x.size()[0], dimension)
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension)
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xb1 = torch.rand(Nb, dimension)
    xb1[:, 0] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension)
    xb2[:, 0] = torch.ones(Nb)
    xb3 = torch.rand(Nb, dimension)
    xb3[:, 1] = torch.zeros(Nb)
    xb4 = torch.rand(Nb, dimension)
    xb4[:, 1] = torch.ones(Nb)
    part_2 = torch.sum((model(xb1) - u_ex(xb1))**2)/x.size()[0] + torch.sum((model(xb2) - u_ex(xb2))**2)/x.size()[0] + torch.sum((model(xb3) - u_ex(xb3))**2)/x.size()[0] + torch.sum((model(xb4) - u_ex(xb4))**2)/x.size()[0]
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 4

In [9]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x

In [10]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [11]:
model = DeepRitzNet(dimension, 4)
model = initparam(model, 0.5)

In [12]:
import torch.optim as optim
import torch.nn as nn
import time

In [13]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [14]:
traintime = 10000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [15]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_2D_Dirichlet.npy", error_save)
np.save("DRM_relative_error_2D_Dirichlet.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(32.9370)
current relative error is:  tensor(1.9566)
current epoch is:  50
current loss is:  tensor(15.7770)
current relative error is:  tensor(1.3809)
current epoch is:  100
current loss is:  tensor(8.5846)
current relative error is:  tensor(1.1874)
current epoch is:  150
current loss is:  tensor(5.9223)
current relative error is:  tensor(0.9853)
current epoch is:  200
current loss is:  tensor(3.8384)
current relative error is:  tensor(0.9062)
current epoch is:  250
current loss is:  tensor(0.5333)
current relative error is:  tensor(0.8032)
current epoch is:  300
current loss is:  tensor(-4.4764)
current relative error is:  tensor(0.5872)
current epoch is:  350
current loss is:  tensor(-8.0999)
current relative error is:  tensor(0.2295)
current epoch is:  400
current loss is:  tensor(-8.5761)
current relative error is:  tensor(0.1756)
current epoch is:  450
current loss is:  tensor(-8.2688)
current relative error is:  tensor(0.1657)
current

current epoch is:  4100
current loss is:  tensor(-9.9667)
current relative error is:  tensor(0.0376)
current epoch is:  4150
current loss is:  tensor(-9.8770)
current relative error is:  tensor(0.0455)
current epoch is:  4200
current loss is:  tensor(-9.7186)
current relative error is:  tensor(0.0326)
current epoch is:  4250
current loss is:  tensor(-10.0062)
current relative error is:  tensor(0.0343)
current epoch is:  4300
current loss is:  tensor(-9.6352)
current relative error is:  tensor(0.0352)
current epoch is:  4350
current loss is:  tensor(-9.6374)
current relative error is:  tensor(0.0360)
current epoch is:  4400
current loss is:  tensor(-9.1291)
current relative error is:  tensor(0.0357)
current epoch is:  4450
current loss is:  tensor(-9.8366)
current relative error is:  tensor(0.0319)
current epoch is:  4500
current loss is:  tensor(-9.7309)
current relative error is:  tensor(0.0353)
current epoch is:  4550
current loss is:  tensor(-9.5378)
current relative error is:  tens

current epoch is:  8150
current loss is:  tensor(-9.7681)
current relative error is:  tensor(0.0145)
current epoch is:  8200
current loss is:  tensor(-9.8049)
current relative error is:  tensor(0.0198)
current epoch is:  8250
current loss is:  tensor(-10.0775)
current relative error is:  tensor(0.0136)
current epoch is:  8300
current loss is:  tensor(-9.9758)
current relative error is:  tensor(0.0191)
current epoch is:  8350
current loss is:  tensor(-10.4617)
current relative error is:  tensor(0.0297)
current epoch is:  8400
current loss is:  tensor(-9.6750)
current relative error is:  tensor(0.0182)
current epoch is:  8450
current loss is:  tensor(-9.4876)
current relative error is:  tensor(0.0350)
current epoch is:  8500
current loss is:  tensor(-8.8815)
current relative error is:  tensor(0.0158)
current epoch is:  8550
current loss is:  tensor(-10.1664)
current relative error is:  tensor(0.0198)
current epoch is:  8600
current loss is:  tensor(-9.7662)
current relative error is:  te

In [16]:
x_test = torch.rand(2, 10)
torch.abs(model(x) - u_ex(x))

tensor([[0.0378],
        [0.0257],
        [0.0089],
        ...,
        [0.0070],
        [0.0399],
        [0.0167]], grad_fn=<AbsBackward>)